In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory 
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth if you want!
# !pip install --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [ ]:
from unsloth import FastLanguageModel  # FastVisionModel for LLMs
import torch
max_seq_length = 2048  # Choose any! We auto support RoPE Scaling internally!
load_in_4bit = True  # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",  # Llama-3.1 2x faster
    "unsloth/Mistral-Small-Instruct-2409",  # Mistral 22b 2x faster!
    "unsloth/Phi-4",  # Phi-4 2x faster!
    "unsloth/Phi-4-unsloth-bnb-4bit",  # Phi-4 Unsloth Dynamic 4-bit Quant
    "unsloth/gemma-2-9b-bnb-4bit",  # Gemma 2x faster!
    "unsloth/Qwen2.5-7B-Instruct-bnb-4bit"  # Qwen 2.5 2x faster!
    "unsloth/Llama-3.2-1B-bnb-4bit",  # NEW! Llama 3.2 models
    "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    "unsloth/Llama-3.2-3B-bnb-4bit",
    "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",
]  # More models at https://docs.unsloth.ai/get-started/all-our-models

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-8B-unsloth-bnb-4bit",
    max_seq_length = max_seq_length,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

In [ ]:
import os
import re
from tqdm import tqdm
import torch
# from transformers import AutoModelForCausalLM, AutoTokenizer

# # Load model and tokenizer using official Qwen3-4B
# # model_name = "Qwen/Qwen3-1.7B"
# # model_name = "Qwen/Qwen3-4B"
# model_name = "unsloth/Qwen3-8B-unsloth-bnb-4bit"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     torch_dtype="auto",
#     device_map="auto"
# )

In [ ]:
model.eval()  # set to eval mode

def solve_geometry_problem(predicates, question, enable_thinking=True):
    """
    Solve geometry problem using Qwen3 Model with step-by-step reasoning

    Args:
        predicates (str): Geometric constraints from image
        question (str): Question to solve
        choices (str): Multiple choice options (A, B, C, D)
        enable_thinking (bool): Enable thinking mode for step-by-step reasoning

    Returns:
        dict: Contains 'thinking_content' and 'content'
    """

    # Enhanced prompt to force choice selection

    prompt = f"""You are an expert AI mathematician specializing in geometry. Your task is to solve the following geometric problem using the provided predicates through systematic reasoning and theorem application.

GIVEN GEOMETRIC PREDICATES:
{predicates}

QUESTION:
{question}

YOUR TASK:
Provide a complete step-by-step solution following the structured approach below, then provide your final answer in proper mathematical LaTeX format.

STEP-BY-STEP SOLUTION PROCESS:

STEP 1: PREDICATE ANALYSIS AND SETUP
- Parse and categorize the given predicates into:
  * Geometric shapes (points, lines, circles, triangles, etc.)
  * Measurements and equalities (lengths, angles, areas)
  * Relationships (perpendicular, parallel, congruent, etc.)
  * Positioning (points on lines/circles, intersections, etc.)
- Identify what specific value or measurement the question is asking for
- Note any special geometric constructions or theorems that might apply

STEP 2: CONSTRAINT SYNTHESIS
- Combine related predicates to understand the complete geometric picture
- Identify key relationships that will be useful for solving
- Look for:
  * Equal lengths or angles that can be substituted
  * Perpendicular relationships that create right triangles
  * Circle properties (radii, chords, central/inscribed angles)
  * Congruent or similar triangles
  * Theorem applications (Pythagorean, inscribed angle, etc.)

STEP 3: SOLUTION STRATEGY
- Based on the predicates and question, determine the most direct solution path
- Identify which geometric theorems, properties, or formulas to apply
- Plan the sequence of logical steps needed to reach the answer

STEP 4: MATHEMATICAL DERIVATION
- Execute your solution strategy step by step
- Show all calculations clearly with proper mathematical notation
- Apply geometric theorems and properties systematically
- Use the relationships established in the predicates
- Substitute known values and solve for unknowns

STEP 5: VERIFICATION AND FINAL ANSWER
- Verify your calculated result makes geometric sense
- Express your final answer in proper mathematical LaTeX format
- Ensure units are included when applicable
- Round to appropriate precision when necessary

GEOMETRIC REASONING GUIDANCE:
- Consider all relevant geometric theorems and properties
- Apply circle, triangle, quadrilateral, and angle theorems as appropriate
- Look for relationships between shapes, measurements, and positions
- Use both basic and advanced geometric principles as needed

PREDICATE USAGE GUIDANCE:
- Interpret predicates based on their geometric meaning and context
- Combine multiple predicates to understand complex relationships
- Consider both direct and derived information from predicate combinations

CRITICAL INSTRUCTIONS:
1. **USE THE PREDICATES SYSTEMATICALLY** - Every predicate provides important information
2. **APPLY RELEVANT GEOMETRIC KNOWLEDGE** - Use any geometric theorems, properties, or principles that help solve the problem
3. **REASON FLEXIBLY** - Adapt your approach based on the specific problem and predicates
4. **SHOW ALL WORK** - Make your reasoning clear and mathematical
5. **BE PRECISE** - Use exact values when possible, approximate only when necessary

⚠️ CRITICAL OUTPUT FORMAT REQUIREMENT ⚠️
YOU MUST END YOUR RESPONSE WITH YOUR FINAL ANSWER IN PROPER LATEX FORMAT.

FORMAT EXAMPLES FOR DIFFERENT ANSWER TYPES:
- Coordinates: $(2,-2)$ or $(0, 0)$
- Angles with degrees: $230^\\circ$ or $319^\\circ$
- Measurements with units: $4.4 \\text{{m}}$ or $85$
- Areas: $\\text{{Area}} = 347.4248\\pi \\text{{cm}}^2$ or $\\text{{Area}} = 113.1 \\text{{cm}}^2$
- Surface Areas: $\\text{{Surface Area}} = 9236.28 \\text{{m}}^2$
- Volumes: $\\text{{Volume}} = 113.10 \\text{{cm}}^3$
- Polar coordinates: $(x,y) = (270^\\circ, 5)$ or $(x,y) = (90^\\circ, 5)$
- Piecewise functions: When $x \\leq -3$, $f(x) = -x-5$; when $x > 3$, $f(x) = x+1$
- Function notation: $g(x) = (x + 4)^2 - 5$
- Domain and Range: $\\text{{Domain: }} [-4, 4], \\quad \\text{{Range: }} [0, 2]$
- Constants: $\\pi$

FORMATTING GUIDELINES:
- Use proper LaTeX syntax with $ symbols for inline math
- Include units when applicable using \\text{{}} for unit labels
- Use appropriate mathematical notation (^\\circ for degrees, \\pi for pi, etc.)
- For areas, volumes, and surface areas, clearly label what the measurement represents
- For coordinates, use parentheses: $(x,y)$
- For ranges and domains, use brackets and interval notation
- Round decimal answers to appropriate precision (typically 1-4 decimal places)
- Use \\quad for spacing between multiple parts of an answer

✅ REQUIRED FORMAT:
Your final answer must be a single mathematical expression in proper LaTeX format, clearly representing the solution to the problem.

🔥 MANDATORY: Your response must end with exactly this format:
Final Answer: $your_latex_expression_here$

Begin your analysis now and remember to end with your final answer in the exact format shown above.
"""

    # Create messages
    messages = [
        {"role": "user", "content": prompt}
    ]

    # Apply chat template with thinking mode
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=enable_thinking
    )

    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    with torch.no_grad():
        # Generate response with official Qwen3 parameters
        if enable_thinking:
            # For thinking mode: Temperature=0.6, TopP=0.95, TopK=20, MinP=0
            generated_ids = model.generate(
                **model_inputs,
                max_new_tokens=10000,  # Adjusted from 15768 to 4000
                temperature=0.6,
                top_p=0.95,
                top_k=20,
                min_p=0.0,
                do_sample=True,
                repetition_penalty=1.1  # To reduce repetitions
            )
        else:
            # For non-thinking mode: Temperature=0.7, TopP=0.8, TopK=20, MinP=0
            generated_ids = model.generate(
                **model_inputs,
                max_new_tokens=10000,  # Adjusted from 15768 to 4000
                temperature=0.7,
                top_p=0.8,
                top_k=20,
                min_p=0.0,
                do_sample=True,
                repetition_penalty=1.1  # To reduce repetitions
            )
    torch.cuda.empty_cache()

    output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist()

    # Improved thinking content parsing
    if enable_thinking:
        # First decode the entire output
        full_output = tokenizer.decode(output_ids, skip_special_tokens=True).strip("\n")
        
        # Try to find thinking tags in the decoded text
        if "<think>" in full_output and "</think>" in full_output:
            # Split by thinking tags
            parts = full_output.split("<think>", 1)
            if len(parts) > 1:
                thinking_part = parts[1].split("</think>", 1)
                if len(thinking_part) > 1:
                    thinking_content = thinking_part[0].strip()
                    content = thinking_part[1].strip()
                else:
                    # No closing think tag found
                    thinking_content = thinking_part[0].strip()
                    content = ""
            else:
                thinking_content = ""
                content = full_output
        else:
            # No thinking tags found, try token-based parsing as fallback
            try:
                # Get the token ID for </think>
                think_end_token = tokenizer.convert_tokens_to_ids("</think>")
                if think_end_token in output_ids:
                    index = len(output_ids) - output_ids[::-1].index(think_end_token)
                    thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
                    content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")
                else:
                    # No thinking token found, treat entire output as content
                    thinking_content = ""
                    content = full_output
            except (ValueError, KeyError):
                # Fallback: treat entire output as content
                thinking_content = ""
                content = full_output
    else:
        # No thinking mode, entire output is content
        thinking_content = ""
        content = tokenizer.decode(output_ids, skip_special_tokens=True).strip("\n")

    return {
        'thinking_content': thinking_content,
        'content': content
    }

In [ ]:
def extract_answer_letter(content):
    """
    Simple function to extract the final mathematical answer from model output.
    Expects the last line to be "Final Answer: $...$"
    """
    
    # Split content into lines and get the last non-empty line
    lines = [line.strip() for line in content.strip().split('\n') if line.strip()]
    
    if not lines:
        return ""
    
    # Check the last few lines for "Final Answer:" pattern (sometimes there's extra whitespace)
    for i in range(min(3, len(lines))):
        line = lines[-(i+1)]
        
        # Look for "Final Answer:" variations
        final_answer_patterns = [
            r'Final Answer:\s*(.+)',
            r'The Final Answer is:\s*(.+)',
            r'The correct answer is:\s*(.+)',
            r'Answer:\s*(.+)',
            r'The answer is:\s*(.+)',
            r'Therefore,?\s*(.+)',
            r'Thus,?\s*(.+)',
            r'So,?\s*(.+)',
        ]
        
        for pattern in final_answer_patterns:
            match = re.search(pattern, line, re.IGNORECASE)
            if match:
                answer = match.group(1).strip()
                return clean_and_format_answer(answer)
    
    # Fallback: look for any math expression in the last line
    last_line = lines[-1]
    
    # Check for various math formats
    math_patterns = [
        r'\$([^$]+)\$',                    # $expression$
        r'\\boxed\{([^}]+)\}',            # \boxed{expression}
        r'\$\$([^$]+)\$\$',               # $$expression$$
        r'\\begin\{align\}(.+?)\\end\{align\}',  # align environment
    ]
    
    for pattern in math_patterns:
        match = re.search(pattern, last_line, re.DOTALL)
        if match:
            return f"${match.group(1).strip()}$"
    
    # Check if the last line looks like a mathematical expression
    if is_mathematical_expression(last_line):
        return f"${last_line}$"
    
    return ""


def clean_and_format_answer(answer):
    """
    Clean and format the extracted answer
    """
    answer = answer.strip()
    
    # Remove trailing punctuation
    answer = re.sub(r'[.,;!?]+$', '', answer)
    
    # If already in LaTeX format, return as is
    if answer.startswith('$') and answer.endswith('$'):
        return answer
    
    # Handle boxed expressions
    if '\\boxed{' in answer:
        boxed_match = re.search(r'\\boxed\{([^}]+)\}', answer)
        if boxed_match:
            return f"${boxed_match.group(1)}$"
    
    # Handle display math
    if answer.startswith('$$') and answer.endswith('$$'):
        return f"${answer[2:-2].strip()}$"
    
    # Handle common mathematical expressions
    if is_mathematical_expression(answer):
        return f"${answer}$"
    
    # Handle text answers that might contain math
    if '=' in answer or any(symbol in answer for symbol in ['π', '°', '^', '_', '\\', '(', ')', '[', ']']):
        return f"${answer}$"
    
    # Handle simple numbers
    if re.match(r'^\d+\.?\d*$', answer):
        return f"${answer}$"
    
    # Handle coordinates
    if re.match(r'^\([^)]+\)$', answer):
        return f"${answer}$"
    
    # Default: wrap in LaTeX
    return f"${answer}$"


def is_mathematical_expression(text):
    """
    Check if text looks like a mathematical expression
    """
    # Common mathematical indicators
    math_indicators = [
        r'\d+',                    # digits
        r'[+\-*/=]',               # basic operators
        r'[(){}\[\]]',            # parentheses/brackets
        r'[ππ²³√∞±≠≈≤≥]',           # common symbols
        r'%',                       # percent
        r'e[+-]?\d+',              # scientific notation
        r'\\[a-zA-Z]+',          # LaTeX commands
        r'\^|_',                   # superscript/subscript markers
        r'(sin|cos|tan|cot|sec|csc|log|ln|exp|sqrt|root|lim|sum|prod|int|frac)',
        r'(Area|Volume|Surface|Domain|Range|Radius|Diameter|Circumference)',
        r'[<>]=?',                  # inequalities
        r'∈|∪|∩|∧|∨|∂|∇',           # set/logic/differential operators
        r'⇒|⇔',                     # implication/equivalence
        r'\|',                     # absolute value bars
    ]
    
    # Check if it contains mathematical elements
    for indicator in math_indicators:
        if re.search(indicator, text, re.IGNORECASE):
            return True
    
    # Check if it's mostly mathematical characters
    math_chars = r'[\d\.\s\^\{\}\\\(\)\,\-\+\=\w°πθαβγδλμσφψω]'
    if len(text) > 0 and len(re.findall(math_chars, text)) / len(text) > 0.7:
        return True
    
    return False

In [ ]:
def validate_and_retry_if_needed(predicates, question, max_retries=3):
    """
    Try to get a valid mathematical answer, with retries if needed.
    """
    for attempt in range(max_retries):
        result = solve_geometry_problem(predicates, question, enable_thinking=True)
        content = result['content']
        # print(f"content: {content}\n")
        thinking_content = result['thinking_content']
        answer_letter = extract_answer_letter(content)
        
        if is_valid_mathematical_answer(answer_letter):
            return content, thinking_content, answer_letter
        
        print(f"Attempt {attempt + 1} failed to extract valid mathematical answer")
    
    # If all attempts fail, try one more time with a very direct prompt
    direct_prompt = f"""Given the following geometry problem, solve it step by step and provide your final answer in proper LaTeX mathematical format.

CONSTRAINTS: {predicates}
QUESTION: {question}

⚠️ CRITICAL OUTPUT FORMAT REQUIREMENT ⚠️
YOU MUST END YOUR RESPONSE WITH YOUR FINAL ANSWER IN PROPER LATEX FORMAT.

FORMAT EXAMPLES FOR DIFFERENT ANSWER TYPES:
- Coordinates: $(2,-2)$ or $(0, 0)$
- Angles with degrees: $230^\\circ$ or $319^\\circ$
- Measurements with units: $4.4 \\text{{ m}}$ or $85$
- Areas: $\\text{{Area}} = 347.4248\\pi \\text{{ cm}}^2$ or $\\text{{Area}} = 113.1 \\text{{ cm}}^2$
- Surface Areas: $\\text{{Surface Area}} = 9236.28 \\text{{ m}}^2$
- Volumes: $\\text{{Volume}} = 113.10 \\text{{ cm}}^3$
- Polar coordinates: $(x,y) = (270^\\circ, 5)$ or $(x,y) = (90^\\circ, 5)$
- Piecewise functions: When $x \\leq -3$, $f(x) = -x-5$; when $x > 3$, $f(x) = x+1$
- Function notation: $g(x) = (x + 4)^2 - 5$
- Domain and Range: $\\text{{Domain: }} [-4, 4], \\quad \\text{{Range: }} [0, 2]$
- Constants: $\\pi$

FORMATTING GUIDELINES:
- Use proper LaTeX syntax with $ symbols for inline math
- Include units when applicable using \\text{{}} for unit labels
- Use appropriate mathematical notation (^\\circ for degrees, \\pi for pi, etc.)
- For areas, volumes, and surface areas, clearly label what the measurement represents
- For coordinates, use parentheses: $(x,y)$
- For ranges and domains, use brackets and interval notation
- Round decimal answers to appropriate precision (typically 1-4 decimal places)
- Use \\quad for spacing between multiple parts of an answer

✅ REQUIRED FORMAT:
Your final answer must be a single mathematical expression in proper LaTeX format, clearly representing the solution to the problem.

🔥 MANDATORY: Your response must end with exactly this format:
Final Answer: $your_latex_expression_here$

Begin your analysis now and remember to end with your final answer in the exact format shown above.
"""
    
    messages = [{"role": "user", "content": direct_prompt}]
    text = tokenizer.apply_chat_template(messages,
                                         tokenize=False, 
                                         add_generation_prompt=True, 
                                         enable_thinking=True)
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        # Use thinking mode parameters for the final retry
        generated_ids = model.generate(
            **model_inputs, 
            max_new_tokens=8000, 
            temperature=0.6,
            top_p=0.95,
            top_k=20,
            min_p=0.0,
            do_sample=True,
            repetition_penalty=1.1
        )
    
    output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist()
    final_content = tokenizer.decode(output_ids, skip_special_tokens=True).strip()
    final_letter = extract_answer_letter(final_content)
    
    # Return the best available answer, or a default mathematical expression if extraction fails
    return final_content, "", final_letter if final_letter else get_default_mathematical_answer(question)

def is_valid_mathematical_answer(answer):
    """
    Check if the extracted answer is a valid mathematical expression.
    Args:
        answer (str): The extracted answer
    Returns:
        bool: True if valid, False otherwise
    """
    if not answer or len(answer.strip()) == 0:
        return False
    
    # Check for common mathematical patterns
    mathematical_patterns = [
        # LaTeX Math Blocks
        r'\$[^$]+\$',                                    # Inline math: $...$
        r'\$\$[^$]+\$\$',                               # Display math: $$...$$
        r'\\begin\{[^}]+\}.*?\\end\{[^}]+\}',          # LaTeX environments (matrices, align, etc.)
        
        # Basic Mathematical Expressions
        r'\\frac\{[^}]+\}\{[^}]+\}',                    # Fractions: \frac{a}{b}
        r'\\sqrt(\[[^\]]+\])?\{[^}]+\}',                # Square/nth roots: \sqrt{x}, \sqrt[n]{x}
        r'[a-zA-Z][a-zA-Z0-9]*\^[^{\s]+',              # Simple exponents: x^2, y^10
        r'[a-zA-Z][a-zA-Z0-9]*\^\{[^}]+\}',            # Complex exponents: x^{2n+1}
        r'[a-zA-Z][a-zA-Z0-9]*_[^{\s]+',               # Simple subscripts: x_i, a_n
        r'[a-zA-Z][a-zA-Z0-9]*_\{[^}]+\}',             # Complex subscripts: x_{i,j}
        
        # Integrals and Sums
        r'\\int[_^{}\s\w]*.*?d[a-zA-Z]',                # Integrals: \int_a^b f(x) dx
        r'\\oint[_^{}\s\w]*.*?d[a-zA-Z]',               # Contour integrals
        r'\\sum[_^{}\s\w]*[^\\]*',                      # Summations: \sum_{i=1}^n a_i
        r'\\prod[_^{}\s\w]*[^\\]*',                     # Products: \prod_{i=1}^n a_i
        r'\\lim[_^{}\s\w]*[^\\]*',                      # Limits: \lim_{x \to 0}
        
        # Functions and Operators
        r'\\(?:sin|cos|tan|sec|csc|cot|arcsin|arccos|arctan)\b',  # Trig functions
        r'\\(?:sinh|cosh|tanh|sech|csch|coth)\b',       # Hyperbolic functions
        r'\\(?:log|ln|exp|lg)\b',                       # Logarithmic/exponential
        r'\\(?:max|min|sup|inf|arg|det|dim|ker|rank)\b', # Mathematical operators
        r'\\(?:gcd|lcm|mod)\b',                         # Number theory functions
        r'\\(?:Re|Im)\b',                               # Real/imaginary parts
        
        # Derivatives and Calculus
        r'\\(?:nabla|partial|grad|div|curl)\b',         # Vector calculus
        r'\\frac\{d[^}]*\}\{d[^}]*\}',                  # Derivatives: dy/dx
        r'\\frac\{\\partial[^}]*\}\{\\partial[^}]*\}',  # Partial derivatives
        r'[a-zA-Z][a-zA-Z0-9]*\'\'*',                   # Prime notation: f', f''
        r'\\dot\{[^}]+\}',                              # Dot notation: \dot{x}
        r'\\ddot\{[^}]+\}',                             # Double dot: \ddot{x}
        
        # Greek Letters and Special Symbols
        r'\\(?:alpha|beta|gamma|delta|epsilon|varepsilon|zeta|eta|theta|vartheta|iota|kappa|lambda|mu|nu|xi|omicron|pi|varpi|rho|varrho|sigma|varsigma|tau|upsilon|phi|varphi|chi|psi|omega|Gamma|Delta|Theta|Lambda|Xi|Pi|Sigma|Upsilon|Phi|Psi|Omega)\b',
        r'\\(?:infty|pi|pm|mp|cdot|times|div|ast|star|circ|bullet|oplus|ominus|otimes|oslash|odot|bigcirc)\b',
        r'\\(?:hbar|ell|wp|partial|nabla|Box|triangle|clubsuit|diamondsuit|heartsuit|spadesuit)\b',
        
        # Relations and Inequalities
        r'\\(?:leq|geq|ll|gg|neq|equiv|approx|cong|simeq|sim|propto|parallel|perp|mid|nmid)\b',
        r'\\(?:subset|supset|subseteq|supseteq|subsetneq|supsetneq|in|notin|ni|owns)\b',
        r'\\(?:cup|cap|setminus|bigcup|bigcap|uplus|sqcup|sqcap|wedge|vee|bigwedge|bigvee)\b',
        r'[<>]=?',                                       # Basic inequalities: <, >, <=, >=
        r'\\(?:leftarrow|rightarrow|leftrightarrow|Leftarrow|Rightarrow|Leftrightarrow|mapsto|hookleftarrow|hookrightarrow)\b',
        
        # Number Systems and Sets
        r'\\mathbb\{[A-Z]\}',                           # Blackboard bold: ℝ, ℂ, ℤ, ℕ, ℚ
        r'\\mathcal\{[A-Z]\}',                          # Calligraphic letters
        r'\\mathfrak\{[a-zA-Z]\}',                      # Fraktur letters
        r'\{[^}]*\}',                                   # Sets: {1,2,3}, {x | x > 0}
        r'\\(?:emptyset|varnothing)\b',                 # Empty set
        
        # Brackets and Delimiters
        r'\\left[\(\[\{|].*?\\right[\)\]\}|]',          # Large delimiters
        r'\\(?:langle|rangle|lceil|rceil|lfloor|rfloor)\b', # Angle brackets, ceiling, floor
        r'\\(?:big|Big|bigg|Bigg)[lr]?[\(\[\{|]',       # Sized delimiters
        
        # Coordinates and Vectors
        r'\([^)]*[-+]?\d+(?:\.\d+)?[^)]*\)',            # Coordinates: (2, -1), (x, y)
        r'\[[^\]]*[-+]?\d+(?:\.\d+)?[^\]]*\]',          # Intervals/vectors: [0,1], [a,b]
        r'\\(?:vec|overrightarrow|overline|underline|hat|tilde|bar|dot|ddot)\{[^}]+\}', # Vector notation
        r'\\(?:binom|choose)\{[^}]+\}\{[^}]+\}',        # Binomial coefficients
        
        # Units and Measurements
        r'\d+\.?\d*\s*(?:mm|cm|m|km|in|ft|yd|mi|kg|g|lb|oz|s|min|h|Hz|°|rad|sr|mol|cd|A|V|W|J|N|Pa|°C|°F|K)\b',
        r'\d+\.?\d*\s*°\s*\w*',                         # Angles: 30°, 45°C
        r'\d+\.?\d*\s*\%',                              # Percentages: 50%, 3.14%
        
        # Scientific Notation and Numbers
        r'[-+]?\d+(?:\.\d+)?[eE][-+]?\d+',              # Scientific notation: 1.23e-4
        r'[-+]?\d*\.\d+',                               # Decimal numbers
        r'[-+]?\d+',                                    # Integers
        
        # Function Definitions and Equations
        r'[a-zA-Z][a-zA-Z0-9]*\([^)]*\)\s*=\s*[^,\n]+', # Function definitions: f(x) = ...
        r'[a-zA-Z][a-zA-Z0-9]*\s*=\s*[^,\n]+',         # Variable assignments: x = ...
        r'(?:Area|Volume|Surface\s*Area|Perimeter|Circumference|Diameter|Radius)\s*=\s*[^,\n]+',
        r'(?:Domain|Range|Codomain)\s*[:=]\s*[^,\n]+',  # Domain/Range specifications
        
        # Logic and Proofs
        r'\\(?:forall|exists|nexists|therefore|because|implies|iff|land|lor|lnot|top|bot)\b',
        r'\\(?:models|vdash|dashv|vDash|Vdash|nvdash|nVdash|nvDash|nVDash)\b',
        r'\\(?:square|blacksquare|qed|QED)\b',          # End of proof symbols
        
        # Probability and Statistics
        r'\\(?:Pr|P|E|Var|Cov|Corr)\b',                # Probability operators
        r'\\(?:mathbb\{E\}|mathbb\{P\}|mathbb\{V\})\b', # Expected value, probability, variance
        r'\\(?:sim|stackrel\{d\}\{=\}|stackrel\{p\}\{\\to\})\b', # Distributions, convergence
        
        # Spacing and Formatting (for completeness)
        r'\\(?:quad|qquad|,|;|:|\!|\\|\s)\b',           # Spacing commands
        r'\\(?:text|mathrm|mathit|mathbf|mathsf|mathtt|mathcal|mathfrak|mathbb)\{[^}]+\}', # Text formatting
    ]
    
    for pattern in mathematical_patterns:
        if re.search(pattern, answer, re.IGNORECASE):
            return True
    
    return False

def get_default_mathematical_answer(question):
    """
    Generate a default mathematical answer based on the question type.
    Args:
        question (str): The original question
    Returns:
        str: A default mathematical expression
    """
    question_lower = question.lower()
    
    if 'coordinate' in question_lower or 'point' in question_lower:
        return "$(0, 0)$"
    elif 'angle' in question_lower or 'degree' in question_lower:
        return "$0^\\circ$"
    elif 'area' in question_lower:
        return "$\\text{Area} = 0 \\text{ cm}^2$"
    elif 'volume' in question_lower:
        return "$\\text{Volume} = 0 \\text{ cm}^3$"
    elif 'surface' in question_lower:
        return "$\\text{Surface Area} = 0 \\text{ cm}^2$"
    elif 'domain' in question_lower:
        return "$\\text{Domain: } [0, 0]$"
    elif 'range' in question_lower:
        return "$\\text{Range: } [0, 0]$"
    elif 'function' in question_lower:
        return "$f(x) = 0$"
    elif 'distance' in question_lower or 'length' in question_lower:
        return "$0 \\text{ units}$"
    else:
        return "$0$"

In [ ]:
if __name__ == "__main__":
    # Input directories
    predicates_dir = "/kaggle/input/MathVerse_IMG5_Gemini/image_folder_5/predicates_output"
    questions_dir = "/kaggle/input/mathverse freeform/mathverse freeform/questions"
    
    # Output directories
    reasoning_output_dir = "/kaggle/working/reasoning_output"
    answer_output_dir = "/kaggle/working/answer_output_qwen"
    os.makedirs(reasoning_output_dir, exist_ok=True)
    os.makedirs(answer_output_dir, exist_ok=True)
    
    # Your target numbers (as integers)
    target_numbers = [327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338,
                      341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352,
                      353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364,
                      365, 366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376,
                      377, 378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388,
                      389, 390, 391, 392, 393, 394, 395, 396, 397, 398, 399, 400,
                      401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412,
                      413, 414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424,
                      425, 426, 427, 428, 431, 432, 433, 434, 435, 436, 437, 438,
                      439, 441, 442, 443, 444, 445, 446, 447, 448, 450, 451, 452,
                      453, 454, 455, 456, 457, 458, 459, 460, 461, 462, 463, 464,
                      465, 467, 468, 469, 470, 471, 472, 473, 474, 475, 481, 482,
                      483, 484, 485, 486, 487, 488, 489, 490, 491, 493, 494, 495,
                      497, 500, 501, 502, 503, 504, 505, 506, 508, 509, 510, 511,
                      512, 513, 514, 515, 519, 520, 522, 529, 530, 531, 532, 533,
                      534, 535, 536, 537, 538, 539, 540, 541, 542, 543, 547, 549,
                      556, 557, 568, 569, 570, 571, 573, 578, 580, 588, 591, 592,
                      593, 595, 596, 599, 600, 601, 602, 604, 605, 606, 608, 610,
                      619, 623, 624, 625, 626, 627, 628, 629, 630, 631, 632, 633,
                      634, 635, 636, 637, 638, 639, 640, 641, 642, 643, 644, 645,
                      646, 647, 648, 649, 650, 651, 652, 653, 654, 655, 656, 657,
                      658, 659, 660, 661, 662, 663, 664, 665, 666, 667, 668, 669,
                      670, 671, 672, 673, 674, 675, 676, 677, 678, 679, 680, 681,
                      682, 683, 684, 685, 686, 687, 688, 689, 690, 691, 692, 693,
                      694, 695, 696, 697, 698, 699, 700, 701, 702, 703, 704, 705,
                      706, 707, 708, 709, 710, 711, 712, 713, 714, 715, 716, 717,
                      718, 719, 720, 721, 722, 723, 724, 725, 726, 727, 728, 729,
                      730, 731, 732, 733, 734, 735, 736, 737, 738, 739, 740, 741,
                      742, 743, 744, 745, 746, 747, 748, 749, 750, 765, 766, 767,
                      770, 771, 775, 776]

    problems_to_solve = target_numbers[50:100]
    # Iterate with progress bar
    for num in tqdm(problems_to_solve):
        num_str = str(num).zfill(3)  # Convert to 3-digit string like '327'
        
        # Paths to input files
        pred_path = os.path.join(predicates_dir, f"{num_str}.txt")
        ques_path = os.path.join(questions_dir, f"{num_str}.txt")
        
        # Check if all required files exist
        if not all(os.path.exists(path) for path in [pred_path, ques_path]):
            print(f"Skipping problem {num_str}: Missing input files")
            continue
        
        # Read inputs
        try:
            with open(pred_path, "r") as f:
                predicates = f.read().strip()
            with open(ques_path, "r") as f:
                question = f.read().strip()
        except Exception as e:
            print(f"Error reading files for problem {num_str}: {e}")
            continue
        
        # Solve with validation and retry mechanism
        simple_content, thinking_content, mathematical_answer = validate_and_retry_if_needed(predicates, question)
        
        # Build the reasoning file content with thinking content included
        reasoning_lines = []
        reasoning_lines.append("=" * 100)
        reasoning_lines.append("PROBLEM DETAILS:")
        reasoning_lines.append("=" * 100)
        reasoning_lines.append(f"PREDICATES:\n{predicates}")
        reasoning_lines.append("")
        reasoning_lines.append(f"QUESTION:\n{question}")
        reasoning_lines.append("")
        
        # Add thinking content if available
        if thinking_content.strip():
            reasoning_lines.append("=" * 100)
            reasoning_lines.append("MODEL'S INTERNAL REASONING (THINKING MODE):")
            reasoning_lines.append("=" * 100)
            reasoning_lines.append(thinking_content)
            reasoning_lines.append("")
        
        reasoning_lines.append("=" * 100)
        reasoning_lines.append("FINAL RESPONSE:")
        reasoning_lines.append("=" * 100)
        reasoning_lines.append(simple_content)
        reasoning_lines.append("")
        reasoning_lines.append("=" * 100)
        reasoning_lines.append(f"EXTRACTED MATHEMATICAL ANSWER: {mathematical_answer}")
        reasoning_lines.append("=" * 100)
        
        reasoning_output = "\n".join(reasoning_lines)
        
        # Write reasoning output to file
        reasoning_out_path = os.path.join(reasoning_output_dir, f"{num_str}.txt")
        with open(reasoning_out_path, "w") as f:
            f.write(reasoning_output)
        
        # Write the mathematical answer to a separate file
        answer_out_path = os.path.join(answer_output_dir, f"{num_str}.txt")
        with open(answer_out_path, "w") as f:
            f.write(mathematical_answer)
        
        print(f"Problem {num_str}: Mathematical Answer = {mathematical_answer}")